In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import requests
from io import StringIO
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

#Import Data
url = 'https://raw.githubusercontent.com/LuizGagine/datasets/main/DADOS_COVID19_MUNICIPIO.csv'
data = requests.get(url).content
df = pd.read_csv(StringIO(data.decode('utf-8')), delimiter=';', decimal=',')

In [ ]:
# Calculate correlation matrix
correlation_matrix = df.corr()
# Plotting correlation matrix in a different style
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', square=True, cmap = 'coolwarm')
plt.title('Correlation matrix of all variables')
plt.show()

In [ ]:
#Select best variables based on influence
selected_df = df[['COD_MUNICIPIO','TX_CASOS_CONFIRMADOS', 'TX_OBITO', 'TX_VACINA_DOSE_ADICIONAL', 'TX_LEITOS_TOTAL', 'IDHM', 'IDHM_E', 'RENDA_PER_CAPITA', 'INDICE_GINI', 'IDHM_L', 'PORCENTAGEM_POPULACAO_MASCULINA', 'TX_VACINA_DOSE_REFORCO_3', 'TX_VACINA_DOSE_REFORCO_1', 'TX_LEITOS_NAO_SUS', 'IDHM_R']]

In [ ]:
# Calculate correlation matrix for the selected variables
selected_correlation_matrix = selected_df.corr()
# Plotting correlation matrix
plt.figure(figsize=(20, 20))
sns.heatmap(selected_correlation_matrix, annot=True, fmt='.2f', square=True, cmap = 'coolwarm')
plt.title('')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# Standardize the data
scaler = StandardScaler()
scaled_selected_df = scaler.fit_transform(selected_df)
# Apply PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(scaled_selected_df)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '9'
from sklearn.cluster import KMeans
# Calculate the sum of squared distances for different numbers of clusters
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k, n_init=10)
    km = km.fit(principalDf)
    sum_of_squared_distances.append(km.inertia_)
# Plot the elbow curve
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=0, n_init=10).fit(principalDf)
# Add the cluster labels to the original DataFrame
selected_df['Cluster'] = kmeans.labels_

In [ ]:
# Calculate the mean values for each cluster
cluster_means = selected_df.groupby('Cluster').mean()
# Identify the cluster with the highest incidence of COVID cases and deaths
highest_incidence_cluster = cluster_means['TX_CASOS_CONFIRMADOS'].idxmax()
# Display the mean values for the cluster with the highest incidence
highest_incidence_cluster

In [ ]:
cluster_means.loc[highest_incidence_cluster]

In [ ]:
# Display the mean values for the other clusters
cluster_means.drop(highest_incidence_cluster)

In [ ]:
# Plotting the clusters
plt.figure(figsize=(10, 10))
scatter = plt.scatter(principalDf['principal component 1'], principalDf['principal component 2'], c=selected_df['Cluster'], cmap='viridis')
plt.title('')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 1')
legend1 = plt.legend(*scatter.legend_elements(), title='Clusters')
plt.add_artist(legend1)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
# Calculate Silhouette Score
silhouette_avg = silhouette_score(principalDf, selected_df['Cluster'])
print('Silhouette Score: ', silhouette_avg)
# Calculate Davies-Bouldin Score
db_score = davies_bouldin_score(principalDf, selected_df['Cluster'])
print('Davies-Bouldin Score: ', db_score)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
def dunn_index(X, labels):
    distances = euclidean_distances(X)
    unique_labels = np.unique(labels)
    inter_cluster_distances = np.inf
    intra_cluster_distances = 0
    for label in unique_labels:
        members = labels == label
        distances_to_cluster = distances[members, :][:, members]
        intra_cluster_distances = max(intra_cluster_distances, distances_to_cluster.max())
        for other_label in unique_labels:
            if label != other_label:
                distances_to_other_cluster = distances[members, :][:, labels == other_label]
                inter_cluster_distances = min(inter_cluster_distances, distances_to_other_cluster.min())
    return inter_cluster_distances / intra_cluster_distances
dunn_index_value = dunn_index(principalDf.values, selected_df['Cluster'].values)
print('Dunn Index: ', dunn_index_value)

In [ ]:
# Create a dataframe with the mean deaths and confirmed cases for each cluster
mean_values_per_cluster = pd.DataFrame({
    'Média de Óbitos': mean_deaths_per_cluster,
    'Média de Casos Confirmados': mean_cases_per_cluster
})

# Plot the bar chart
mean_values_per_cluster.plot(kind='bar', figsize=(10, 7))
plt.xlabel('Cluster')
plt.title('')
plt.show()

In [ ]:
# Create a figure and a set of subplots
fig, ax1 = plt.subplots(figsize=(10, 7))

# Plot the mean deaths on the first y-axis
ax1.bar(mean_values_per_cluster.index, mean_values_per_cluster['Média de Óbitos'], color='b', alpha=0.5, label='Média de Óbitos')
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Média de Óbitos', color='b')
ax1.tick_params('y', colors='b')

# Create a second y-axis
ax2 = ax1.twinx()

# Plot the mean confirmed cases on the second y-axis
ax2.bar(mean_values_per_cluster.index, mean_values_per_cluster['Média de Casos Confirmados'], color='r', alpha=0.5, label='Média de Casos Confirmados')
ax2.set_ylabel('Média de Casos Confirmados', color='r')
ax2.tick_params('y', colors='r')

# Set the title
plt.title('Média de Óbitos e Casos Confirmados por Cluster')

# Show the plot
plt.show()

In [ ]:
# Filter the dataframe for Cluster 0
cluster_0_df = selected_df[selected_df['Cluster'] == 0]

# Calculate the correlation matrix
cluster_0_corr = cluster_0_df.corr()

# Plot the correlation matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cluster_0_corr, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Matriz de Correlação do Cluster 0')
plt.show()

In [ ]:
# Correlation of 'TX_CASOS_CONFIRMADOS' with other variables
confirmed_cases_corr = cluster_0_corr['TX_CASOS_CONFIRMADOS'].sort_values(ascending=False)

# Correlation of 'TX_OBITO' with other variables
deaths_corr = cluster_0_corr['TX_OBITO'].sort_values(ascending=False)

# Create a dataframe with the correlations
correlations_df = pd.DataFrame({
    'Casos Confirmados': confirmed_cases_corr,
    'Óbitos': deaths_corr
})

# Remove the correlations of the variables with themselves
correlations_df = correlations_df[correlations_df.index != 'TX_CASOS_CONFIRMADOS']
correlations_df = correlations_df[correlations_df.index != 'TX_OBITO']

# Display the dataframe
correlations_df